In [1]:
# 1. Import Libraries
import os
import pandas as pd
import numpy as np
import cv2
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import load_model

In [2]:
# 2. Load and Explore the Music Data
music_data = pd.read_csv('data_moods.csv')
print(music_data.head())

                                          name                        album  \
0                                         1999                         1999   
1                                           23                           23   
2                                     9 Crimes                            9   
3                               99 Luftballons               99 Luftballons   
4  A Boy Brushed Red Living In Black And White  They're Only Chasing Safety   

           artist                      id release_date  popularity  length  \
0          Prince  2H7PHVdQ3mXqEHXcvclTB0   1982-10-27          68  379266   
1  Blonde Redhead  4HIwL9ii9CcXpTOTzMq0MP   2007-04-16          43  318800   
2     Damien Rice  5GZEeowhvSieFDiR8fQ2im   2006-11-06          60  217946   
3            Nena  6HA97v4wEGQ5TUClRM0XLc   1984-08-21           2  233000   
4       Underoath  47IWLfIKOKhFnz1FUEUIkE   2004-01-01          60  268000   

   danceability  acousticness  energy  instrumentalness 

In [3]:
# 3. Define Directory Paths
base_dir = 'mood_for_music'
train_dir = os.path.join(base_dir, 'train')
test_dir = os.path.join(base_dir, 'test')

In [4]:
# 4. Image Data Preprocessing
# Create an ImageDataGenerator for training and testing
train_datagen = ImageDataGenerator(rescale=1./255)
test_datagen = ImageDataGenerator(rescale=1./255)

In [5]:
# Create generators for training and testing
train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(48, 48),
    batch_size=32,
    color_mode='grayscale',
    class_mode='categorical'
)

Found 28709 images belonging to 7 classes.


In [6]:
test_generator = test_datagen.flow_from_directory(
    test_dir,
    target_size=(48, 48),
    batch_size=32,
    color_mode='grayscale',
    class_mode='categorical'
)

Found 7178 images belonging to 7 classes.


In [7]:
# 5. Build the CNN Model
model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(48, 48, 1)),
    MaxPooling2D((2, 2)),
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    Conv2D(128, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    Flatten(),
    Dense(128, activation='relu'),
    Dropout(0.5),
    Dense(7, activation='softmax')  # 7 classes for each emotion
])

C:\Users\home\AppData\Roaming\Python\Python311\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [8]:
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])


In [9]:
# 6. Train the Model
history = model.fit(train_generator, validation_data=test_generator, epochs=10)


Epoch 1/10


C:\Users\home\AppData\Roaming\Python\Python311\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:122: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


898/898 ━━━━━━━━━━━━━━━━━━━━ 1184s 1s/step - accuracy: 0.2715 - loss: 1.7810 - val_accuracy: 0.4146 - val_loss: 1.5237
Epoch 2/10
898/898 ━━━━━━━━━━━━━━━━━━━━ 383s 423ms/step - accuracy: 0.4144 - loss: 1.5132 - val_accuracy: 0.4737 - val_loss: 1.3790
Epoch 3/10
898/898 ━━━━━━━━━━━━━━━━━━━━ 83s 93ms/step - accuracy: 0.4752 - loss: 1.3851 - val_accuracy: 0.5017 - val_loss: 1.3055
Epoch 4/10
898/898 ━━━━━━━━━━━━━━━━━━━━ 82s 91ms/step - accuracy: 0.5033 - loss: 1.3083 - val_accuracy: 0.5176 - val_loss: 1.2665
Epoch 5/10
898/898 ━━━━━━━━━━━━━━━━━━━━ 84s 93ms/step - accuracy: 0.5211 - loss: 1.2579 - val_accuracy: 0.5219 - val_loss: 1.2436
Epoch 6/10
898/898 ━━━━━━━━━━━━━━━━━━━━ 84s 93ms/step - accuracy: 0.5465 - loss: 1.1981 - val_accuracy: 0.5380 - val_loss: 1.2070
Epoch 7/10
898/898 ━━━━━━━━━━━━━━━━━━━━ 84s 94ms/step - accuracy: 0.5604 - loss: 1.1669 - val_accuracy: 0.5464 - val_loss: 1.1951
Epoch 8/10
898/898 ━━━━━━━━━━━━━━━━━━━━ 780s 869ms/step - accuracy: 0.5773 - loss: 1.1221 - val_acc

In [10]:
# 7. Save the Model
model.save('mood_classification_model.h5')

In [22]:
# 8. Define a Function to Predict Mood and Recommend Music
def predict_mood_and_recommend_music(image_path):
    # Load and preprocess the image
    image = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)
    image = cv2.resize(image, (48, 48))
    image = np.expand_dims(image, axis=0)
    image = np.expand_dims(image, axis=-1)
    image = image / 255.0  # Normalize the image
     # Load the trained model
    model = load_model('mood_classification_model.h5')
    # Predict the mood
    predicted_class = model.predict(image)
    predicted_mood = np.argmax(predicted_class)
    # Map the predicted mood to the actual labels
    mood_labels = {0: 'angry', 1: 'disgust', 2: 'fear', 3: 'happy', 4: 'neutral', 5: 'sad', 6: 'surprise'}
    predicted_mood_label = mood_labels[predicted_mood]
    # Recommend music based on the predicted mood
    recommendations = music_data[music_data['mood'].str.lower() == predicted_mood_label]
    recommended_songs = recommendations[['name', 'artist']].sample(5)
    
    return predicted_mood_label, recommended_songs
# 9. Example Usage
image_path = "C:\\Users\\home\\Desktop\\Untitled Folder\\mood_for_music\\test\\happy\\PrivateTest_95094.jpg"  # Replace with an actual image path
mood, songs = predict_mood_and_recommend_music(image_path)
print(f"Predicted Mood: {mood}")
print("Recommended Songs:")
print(songs)

1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 814ms/step
Predicted Mood: happy
Recommended Songs:
                  name             artist
162  Edge of Seventeen       Stevie Nicks
587        Tubthumping        Chumbawamba
557          The Joker  Steve Miller Band
372        Night Moves          Bob Seger
502  Spirit In The Sky   Norman Greenbaum
